In [1]:
import math
import cmath
import random

# from sympy import Symbol, re, solve

In [2]:
def solve_cubEq(a,b,c,d):
    i = cmath.sqrt(-1)
    
    p = (c/(3*a)) - ((b**2)/(9*(a**2)))
    q = (d/a) + ((2*(b**3))/(27*(a**3))) - ((c*b)/(3*(a**27)))
    w = (-1 + cmath.sqrt(-3))/2
    
    x1 = -b / (3*a) + ((-q + (q**2 + 4*(p**3))**(1/2))/2)**(1/3) + ((-q - (q**2 + 4*(p**3))**(1/2))/2)**(1/3)
    x2 = -b / (3*a) + (((-q + (q**2 + 4*(p**3))**(1/2))/2)**(1/3))*(w) + (((-q - (q**2 + 4*(p**3))**(1/2))/2)**(1/3))*(w**2)
    x3 = -b / (3*a) + (((-q + (q**2 + 4*(p**3))**(1/2))/2)**(1/3))*(w**2) + (((-q - (q**2 + 4*(p**3))**(1/2))/2)**(1/3))*(w)
    
    res = [x1, x2, x3]
    
#     return list(filter(lambda x: x.imag == 0, res))
    return res

In [3]:
def solve_cubEq_bisect(a,b,c,d):
    def f(x):
        return a*x**3 + b*x**2 + c*x + d
    while True:
        r1= random.randint(-100, 100)
        r2 = random.randint(-100, 100)
        if f(r1)*f(r2) < 0:
            p = r1
            q = r2
            break
    m = p + (q - p)/2
    err = abs(f(m))
#     n=0
    while err > 10**(-4):
#         n = n + 1
        
        if f(m)*f(p) < 0:
            q = m
        else:
            p = m
        m = p + (q-p)/2
        err = abs(f(m))
    
    return round(m,3)

In [4]:

qa = 150 # 허용지내력 kPa
h_BOFDN = 2.4 # 기초 저면 m  ## from revit input
S = 10 # Surcharge Load kN/m^2
w_c = 24 #콘크리트 단중 kN/m^3
w_s = 17 # 흙 단중 kN/m^3
fc = 32 #콘크리트 압축강도MPa  ## from revit input
fy = 420 #철근 항복강도 MPa
Cx = 0.5 #기둥치수 x m  ## from revit input
Cy = 0.8 #기둥치수 y m  ## from revit input
cc = 75 #피복두께 mm  ## from revit input or Excel input


Dc = 16 # 기둥 Pedestal 주근 mm  from revit input or Excel input
D = 16 # 기초 배근 mm

ldb = max(0.25*Dc*fy/(fc)**0.5, 0.043*Dc*fy) #기둥 철근의 압축 정착 길이

Ps = 850  ## from frame analysis module
Pu = 1235.9  ## from frame analysis module
Msx = 150  ## from frame analysis module
Msy = 120  ## from frame analysis module
Mux = 230  ## from frame analysis module
Muy = 207.2  ## from frame analysis module

In [5]:
#흙과 기초 평균 두께에 의한 유효 허용 지내력 산정
qe = qa - (w_c+w_s)/2*h_BOFDN-S 
a = qe
b = 0
c = -Ps
d = -6*Msx-6*Msy

In [6]:
# x = Symbol('x')
# eq = a*x**3+b*x**2+c*x+d
# kk_ = solve(eq)
kk = solve_cubEq(a,b,c,d)
x1 = kk[0]
x2 = kk[1]
x3 = kk[2]

print(x1)
print(x2)
print(x3)

3.756433698771615
(-1.8782168493858074+1.1053787223273466j)
(-1.8782168493858071-1.1053787223273466j)


In [7]:
if round(x1.real,1) > 0:
    Lx = round(x1.real,2)
elif round(x2.real,1) > 0:
    Lx = round(x2.real,2)
elif round(x3.real,1) > 0:
    Lx = round(x3.real,2)

# import math
Lx = max(math.ceil(Lx*10)/10,Cx) 
Ly = max(Lx, Cy)

print("Lx = ", round(Lx,1), "m")
print("Ly = ", round(Ly,1), "m")

Lx =  3.8 m
Ly =  3.8 m


In [8]:
#1면 전단에 의한 두께 산정 d1, d2
d1 = ((Ly/2-Cy/2)*(6*Mux/(Lx*Ly**2)+Pu/(Lx*Ly)))*Lx/(0.75*0.17*fc**0.5*Lx*1000+6*Mux/(Lx*Ly**2)*Lx+Pu/(Lx*Ly)*Lx)
d2 = ((Lx/2-Cx/2)*(6*Muy/(Ly*Lx**2)+Pu/(Lx*Ly)))*Ly/(0.75*0.17*fc**0.5*Ly*1000+6*Muy/(Ly*Lx**2)*Ly+Pu/(Lx*Ly)*Ly)
print(d1)
print(d2)

0.19965110319771873
0.21531714192595733


In [9]:
#2면 전단에 의한 두께 산정 d3
s1 = (Pu)/(Lx*Ly)+Mux/(Lx*Ly**2/6)+Muy/(Ly*Lx**2/6)
s2 = (Pu)/(Lx*Ly)+Mux/(Lx*Ly**2/6)-Muy/(Ly*Lx**2/6)
s3 = (Pu)/(Lx*Ly)-Mux/(Lx*Ly**2/6)-Muy/(Ly*Lx**2/6)
s4 = (Pu)/(Lx*Ly)-Mux/(Lx*Ly**2/6)+Muy/(Ly*Lx**2/6)

s_max = max(s1, s2, s3, s4)
s_min = min(s1, s2, s3, s4)
s_avg = (s_max+s_min)/2

a = s_avg-0.75*0.33*fc**0.5*4*1000
b = (Cx + Cy)*s_avg-0.75*0.33*fc**0.5*2*(Cx+Cy)*1000
c = (Lx*Ly-Cx*Cy)*s_avg

x1 = (-b+(b**2-4*a*c)**0.5)/(2*a)
x2 = (-b-(b**2-4*a*c)**0.5)/(2*a)

if x1 <= 0:
    d3 = x2
elif x2 <= 0:
    d3 = x1

print(d3)

0.24597159542074223


In [10]:
#기초 두께 산정
d4 = max(d1*1000, d2*1000, d3*1000, ldb)
H = math.ceil((d4+cc)/100)*100

print(H)

400


In [11]:
#기초 두께에 따른 허용 지내력 재산정 / 기초 크기 재산정
qe = qa - (H/1000*w_c+(h_BOFDN-H/1000)*w_s*(Lx*Ly-Cx*Cy)/(Lx*Ly)+Cx*Cy/(Lx*Ly)*(h_BOFDN-H/1000)*(w_c))-S
a = qe
b = 0
c = -Ps
d = -6*Msx-6*Msy

# x = Symbol('x')
# eq = a*x**3+b*x**2+c*x+d
kk = solve_cubEq(a,b,c,d)
x1 = kk[0]
x2 = kk[1]
x3 = kk[2]

if round(x1.real,1) > 0:
    Lx = round(x1.real,2)
elif round(x2.real,1) > 0:
    Lx = round(x2.real,2)
elif round(x3.real,1) > 0:
    Lx = round(x3.real,2)

# import math
Lx = max(math.ceil(Lx*10)/10,Cx) 
Ly = max(Lx, Cy) 
# print("")

In [12]:
#편심 제한 기준
while Lx/6 < Msy/Ps:
    Lx = Lx + 0.1
print ("Lx = ", round(Lx,1), "m")

while Ly/6 < Msx/Ps:
    Ly = Ly + 0.1
print ("Ly = ", round(Ly,1), "m")

Lx =  3.7 m
Ly =  3.7 m


In [13]:
#1면 전단에 의한 두께 재산정 d1, d2
d1 = ((Ly/2-Cy/2)*(6*Mux/(Lx*Ly**2)+Pu/(Lx*Ly)))*Lx/(0.75*0.17*fc**0.5*Lx*1000+6*Mux/(Lx*Ly**2)*Lx+Pu/(Lx*Ly)*Lx)
d2 = ((Lx/2-Cx/2)*(6*Muy/(Ly*Lx**2)+Pu/(Lx*Ly)))*Ly/(0.75*0.17*fc**0.5*Ly*1000+6*Muy/(Ly*Lx**2)*Ly+Pu/(Lx*Ly)*Ly)

print(d1)
print(d2)

0.20316227530054454
0.21973479457383271


In [14]:
#2면 전단에 의한 두께 재산정 d3
s1 = (Pu)/(Lx*Ly)+Mux/(Lx*Ly**2/6)+Muy/(Ly*Lx**2/6)
s2 = (Pu)/(Lx*Ly)+Mux/(Lx*Ly**2/6)-Muy/(Ly*Lx**2/6)
s3 = (Pu)/(Lx*Ly)-Mux/(Lx*Ly**2/6)-Muy/(Ly*Lx**2/6)
s4 = (Pu)/(Lx*Ly)-Mux/(Lx*Ly**2/6)+Muy/(Ly*Lx**2/6)

s_max = max(s1, s2, s3, s4)
s_min = min(s1, s2, s3, s4)
s_avg = (s_max+s_min)/2

a = s_avg-0.75*0.33*fc**0.5*4*1000
b = (Cx + Cy)*s_avg-0.75*0.33*fc**0.5*2*(Cx+Cy)*1000
c = (Lx*Ly-Cx*Cy)*s_avg

x1 = (-b+(b**2-4*a*c)**0.5)/(2*a)
x2 = (-b-(b**2-4*a*c)**0.5)/(2*a)

if x1 < 0:
    d3 = x2
elif x2 < 0:
    d3 = x1


print ("")
d4 = max(d1*1000, d2*1000, d3*1000, ldb)
print ("Required FDN Effective Depth = ", round(d4,1), "mm")
H = math.ceil((d4+cc+D)/100)*100
print ("FDN Full Depth = ", H, "mm")
print ("")


Required FDN Effective Depth =  297.0 mm
FDN Full Depth =  400 mm



In [15]:
#요구모멘트산정
Mux1 = (Ly/2-Cy/2)**2/2*(Pu/(Lx*Ly)+abs(Mux/(Lx*Ly**2/6)))
Muy1 = (Lx/2-Cx/2)**2/2*(Pu/(Lx*Ly)+abs(Muy/(Ly*Lx**2/6)))
print ("Mux =", round(Mux1,2), "kNm/m")
print ("    = ", round(Mux1 * Lx,2), "kNm")
print ("Muy =", round(Muy1,2), "kNm/m")
print ("    = ", round(Muy1 * Ly,2), "kNm")

Mux = 123.54 kNm/m
    =  457.12 kNm
Muy = 146.97 kNm/m
    =  543.79 kNm


In [16]:
#방향별 모멘트 크기에 따른 주근과 부근 유효 깊이 산정
if Mux1 < Muy1:
    dx = H-cc-3/2*D
    dy = H-cc-1/2*D
else:
    dy = H-cc-3/2*D
    dx = H-cc-1/2*D

print ("")
print ("dx = ", round(dx, 4), "mm")
print ("dy = ", round(dy, 4), "mm")
print ("")


dx =  301.0 mm
dy =  317.0 mm



In [17]:
#각 방향별 휨 철근 설계
b = Lx*1000
d = dx
Mu = Mux1*10**6

a1 = ((-0.9*0.85*fc*b*d)+((0.9*0.85*fc*b*d)**2-4*(-0.9*0.85*fc*b*0.5)*(-Mu))**0.5)/(2*(-0.9*0.85*fc*b*0.5))
a2 = ((-0.9*0.85*fc*b*d)-((0.9*0.85*fc*b*d)**2-4*(-0.9*0.85*fc*b*0.5)*(-Mu))**0.5)/(2*(-0.9*0.85*fc*b*0.5))

if a1 < 0:
    a = a2
elif a2 < 0:
    a = a1
elif a1 > d:
    a = a2
elif a2 > d:
    a = a1

Asx = max(0.85*fc*a*b/fy, 0.0018*1000*H)
print ("Asx = ", round(Asx,1), "mm2/m")
print ("    = ", round(Asx,1)* Lx, "mm2")
sx = int(1000/(Asx/((D/2)**2*3.141))/50)*50
print ("D",D,"@",sx)
print ("")

b = Ly*1000
d = dy
Mu = Muy1*10**6

a1 = ((-0.9*0.85*fc*b*d)+((0.9*0.85*fc*b*d)**2-4*(-0.9*0.85*fc*b*0.5)*(-Mu))**0.5)/(2*(-0.9*0.85*fc*b*0.5))
a2 = ((-0.9*0.85*fc*b*d)-((0.9*0.85*fc*b*d)**2-4*(-0.9*0.85*fc*b*0.5)*(-Mu))**0.5)/(2*(-0.9*0.85*fc*b*0.5))

if a1 < 0:
    a = a2
elif a2 < 0:
    a = a1
elif a1 > d:
    a = a2
elif a2 > d:
    a = a1

Asy = max(0.85*fc*a*b/fy, 0.0018*1000*H)
print ("Asy = ", round(Asy,1), "mm2/m")
print ("    = ", round(Asy,1) * Ly, "mm2")
sy = int(1000/(Asy/((D/2)**2*3.141592))/50)*50
print ("D",D,"@",sy)

Asx =  1094.1 mm2/m
    =  4048.17 mm2
D 16 @ 150

Asy =  1236.6 mm2/m
    =  4575.42 mm2
D 16 @ 150


In [18]:
#허용지내력 만족 여부 검토
W_c = (Lx*Ly*H/1000+Cx*Cy*(h_BOFDN-H/1000))*w_c
W_s = (Lx*Ly-Cx*Cy)*(h_BOFDN-H/1000)*w_s

a1 = (Ps+W_c+W_s)/(Lx*Ly)+Msx/(Lx*Ly**2/6)+Msy/(Ly*Lx**2/6)+S
a2 = (Ps+W_c+W_s)/(Lx*Ly)+Msx/(Lx*Ly**2/6)-Msy/(Ly*Lx**2/6)+S
a3 = (Ps+W_c+W_s)/(Lx*Ly)-Msx/(Lx*Ly**2/6)-Msy/(Ly*Lx**2/6)+S
a4 = (Ps+W_c+W_s)/(Lx*Ly)-Msx/(Lx*Ly**2/6)+Msy/(Ly*Lx**2/6)+S

print ("")
print ("Service Soil Pressure = ", round(max(a1, a2, a3, a4),2), "kPa")
print ("Factored Soil Pressure = ", round(max(s1, s2, s3, s4),2), "kPa")
print ("")


Service Soil Pressure =  148.08 kPa
Factored Soil Pressure =  142.07 kPa



In [19]:
#1방향/2방향 전단검토
dxy = (dx+dy)/2

Vux = (Ly/2-Cy/2-dx/1000)*(max(Mux/(Lx*Ly**2/6),-Mux/(Lx*Ly**2/6))+Pu/(Lx*Ly))*Lx
print ("Vux =" , round(Vux,2), "kN")
phiVcx = 0.75*0.17*(fc)**0.5*Lx*1000*dx/1000
print("phiVcx =" , round(phiVcx,2), "kN")
print ("")

Vuy = (Lx/2-Cx/2-dy/1000)*(max(Muy/(Ly*Lx**2/6),-Muy/(Ly*Lx**2/6))+Pu/(Lx*Ly))*Ly
print ("Vuy =", round(Vuy,2), "kN")
phiVcy = 0.75*0.17*(fc)**0.5*Ly*1000*dy/1000
print("phiVcy =", round(phiVcy,2), "kN")
print ("")

Vup = (Lx*Ly-(Cx+dy/1000)*(Cy+dx/1000))*s_avg
print ("Vup =", round(Vup,2), "kN")
phiVcp = 0.75*0.33*(fc)**0.5*(2*Cx+2*Cy+4*dxy/1000)*dxy/1000*1000
print ("phiVcp =", round(phiVcp,2), "kN")
print ("")

Vux = 499.62 kN
phiVcx = 803.25 kN

Vuy = 545.07 kN
phiVcy = 845.95 kN

Vup = 1154.69 kN
phiVcp = 1659.54 kN

